# Automation

## Introduction

Condition Based Monitoring, sometimes called Edge Network Automation, is the idea that edge sensors and actuators should not be responsible for decision making or coordinating the responses to various edge network condition. However, neither should all of this decision making and analytics processing be performed in the cloud. The IoT Gateway should be remotely programmable by an IoT administrator or software developer in such a way that local events can be automatically managed and reported. Intelligent closed loop systems are able to coordinate responses to conditions on their own networks and report back to the cloud or a network operation center.

In the example of a temperature controlled room, the temperature sensor is reporting the temperature to the Intel® IoT Gateway and the gateway is responsible for triggering an IoT event. An IoT Event (also called a trigger in this workshop) always has a conditions function and a trigger function. In the case that the temperature is too hot the event may take automatic action to turn on the air conditioning on and send an alert to the person responsible for the room.



## Purpose of the Lab

In this lab, we will implement a Python based rules engine to automate IoT action on a network. This system can be used to build a smart home, a smart factory or other Internet of Things installations.

The student should note that this implementation's primary goal is to teach the fundementals of implementing a rules based IoT automation system. 

There are many other considerations in building a more production ready environment. A production environment should consider the actors of the system such as the system administrators, business policy makers, factory workers, developers, etc ... It should also consider integration with the pre-existing software environment, enhanced security measures and building a human interface device and graphical user interface for the operators.

This lab's purpose is to focus on the core ideas and not to distract with the many ways that it could be built out or integrated into other enviroments.

## Objectives

By the end of this module, you should be able to:

* Implement your automation rules in the form of conditional actions called triggers that run on the edge network.
* Read triggers from the database and evaluate whether any of them should be executed.
* Write your own Automation Service




## Prerequisites and Architecture Overview

As we begin this lab, we assume that the developer has an Intel IoT device on the network and that there are **sensors on the network transmitting data over MQTT**.

Our automation service will start by reading in a list of automation rules and listening to incoming MQTT sensor data.

A Rule is defined to have a Name, and Sensor that it monitors, a true/false predicate function to determine whether the rule should be run and a function action to run if the predicate is true.

To build this automation service, we will start by: 
1. **Defining a Rule Class** and creating several examples to get the student familiar with them. 
2. Secondly, we will **define several helper functions** that will allow us to use a function programming style to filter the list of Rules. When data comes in to the automation service, we will want to filter the list of Rules so that only rules that apply to incoming data are checked and evaluated.
3. **Create a simple Python service to listen for MQTT**
4. Use the **helper functions to filter the list of rules**.
5. **Execute every rule whose predicate function evaluates to true**.


## Implementing a Rule

### The Rule Class

Here is the definition of a Rule base class. To define a rule you will need to extend this class. 

This base class provides:
* a **constructor function** that assigns the rule to a unique sensor on the network. 
* default **predicate function** which returns always False. This means that by default the action function is not never called. This function should be overridden in a derived class.
* an **__do function** that calls the predicate function and if it returns true then calls the action function
* an **action function** to perform the main task. This function should be overridden in a derived class.

Note that the __\_\_do function__ is a private function that can not be accessed outside of the class.

In [1]:
class Rule:
    """
    A Base Class for defining IoT automation rules.
    """
    
    def __init__(self, sensorID):
        """
        Constructor function that takes a id 
        that uniquely identifies a sensor.
        """
        self.sensorID = sensorID
    
    def predicate(self, sensorValue):
        "In the base Rule class, the predicate always returns False"
        return False
                    
    def action(self, sensorValue):
        print("Generic Rule activiation on " + 
              self.sensorID + " with senor value " + str(sensorValue))    

### Define a Subclass Rule Class that can be Instantiated

Now that we've defined the base Rule class, we can show examples of how to use it to derive a specific rule class.

The sky is the limit with the action that you choose to define. Some common action might be to sent a text message to a list of system administrators when something isn't working properly, or to log a message to a database. You can include any Python libraries that you wish when defining these functions.

Let define a simple **Rule that prints to the console when the temperature rises over 25C**.

In [2]:
class TemperatureOver25(Rule):
    def predicate(self, sensorValue):
        return sensorValue > 25
    
    def action(self, sensorValue):
        print("Temperature Over 25 Rule activated on " + self.sensorID + " with senor value " + str(sensorValue))

And another rule that **prints to the console if the temperature falls below 20C**.

In [3]:
class TemperatureUnder20(Rule):
    def predicate(self, sensorValue):
        return sensorValue < 20
    
    def action(self, sensorValue):
        print("Temperature Under 20 Rule activated on " + 
              self.sensorID + " with senor value " + str(sensorValue))
        

And another rule that **prints to the console if the rotary angle sensor is over 500 **.

In [4]:
class RotaryAngleOver500(Rule):
    def predicate(self, sensorValue):
        return sensorValue > 500
    
    def action(self, sensorValue):
        print("Rotary Angle Sensor over 500. Rule activated on " + 
              self.sensorID + " with senor value " + str(sensorValue))
        

### Instantiate the Rules

Each rule is instantiated by passing in a temperature sensor that the rule monitors. Then we will add the rules to a Python list and displaying them. A list will allow us to use functional programming to filter and execute the rules.

In [5]:
r1 = Rule("generic_Sensor")  # Generic Rule assigned to a sensor name "generic_Sensor"
r2 = TemperatureOver25("temperature") # Derived rule assigned to a sensor name "temperature"
r3 = TemperatureUnder20("temperature") # Derived rule assigned to a sensor name "temperature"
r4 = RotaryAngleOver500("rotaryAngle") # RotaryAngleOver500 assigned to a sensor name "rotaryAngle"

In [6]:
rules = [r1, r2, r3, r4]
rules

[<__main__.Rule instance at 0x104eb9320>,
 <__main__.TemperatureOver25 instance at 0x104eae8c0>,
 <__main__.TemperatureUnder20 instance at 0x104eb9368>,
 <__main__.RotaryAngleOver500 instance at 0x104eb9170>]

### Define Helper Functions

Theses helper functions will take the entire list of automation rules and filter it based on sensorID. Becasue we are listening to JSON object that contain a **sensorID**, a **value** and a **timestamp**, this enables us to filter the automation rules to only include the rules that apply to the sensorID of the incoming data.

In [7]:
def filterBySensorId(sensorID, rules):
    "Filter a list of rules by sensorID"
    return [rule for rule in rules if rule.sensorID == sensorID]

The second filter that we will define will take a list of automation rules determine if there **predicate** functions will evaluate to **True** based on the incoming data.

In [8]:
def evalPredicate(sensorData, rules):
    "Filter a list of rules by its predicate"
    [rule for rule in rules if rule.predicate(sensorData) == True]

### Test the Helper Functions

The ***filterBySensorId*** function takes the list of all rules and returns the rules associated with a given sensor. In this case, we will filter rules by the sensor named "temperature."  There will be two rules associated with "temperature".

In [9]:
filteredRules = filterBySensorId("temperature", rules)
print filteredRules

[<__main__.TemperatureOver25 instance at 0x104eae8c0>, <__main__.TemperatureUnder20 instance at 0x104eb9368>]


The ***evalPredicate*** function takes the list of all rules and evaluates the rules predicate function. It returns a list of booleans for each rule function. Let's pass in a sensor data point of 10 degrees. Only the ***TemperatureUnder15Rule*** will return **True**, and therefore it is the only rule that should execute.

In [10]:
[[r, r.predicate(10)] for r in filteredRules] # Only the TemperatureUnder15Rule rule will evaluate to True and execute

[[<__main__.TemperatureOver25 instance at 0x104eae8c0>, False],
 [<__main__.TemperatureUnder20 instance at 0x104eb9368>, True]]

## Declare Sensor Data JSON Objects

Next we will setup the JSON library and declare a temperature JSON object. This JSON format is the same JSON format that you will see from the **sensor labs** and **virtual sensor tool**.

In [11]:
import json, sys

Let's declare a couple of sensor data JSON objects.

In [12]:
# Declare a sample piece of data that comes from the temperature sensor
high_rotaryAngle_String = '''
{
  "sensor_id":"rotaryAngle",
  "value":600,
  "timestamp":1513807710949
}
'''

high_temperature_String = '''
{
  "sensor_id":"temperature",
  "value":40,
  "timestamp":1513807710949
}
'''

low_temperature_String = '''
{
  "sensor_id":"temperature",
  "value":10,
  "timestamp":1513807710949
}
'''
print("Three JSON Sensor Data Strings")
print("==============================")
print(high_rotaryAngle_String)
print(high_temperature_String)
print(low_temperature_String)

Three JSON Sensor Data Strings

{
  "sensor_id":"rotaryAngle",
  "value":600,
  "timestamp":1513807710949
}


{
  "sensor_id":"temperature",
  "value":40,
  "timestamp":1513807710949
}


{
  "sensor_id":"temperature",
  "value":10,
  "timestamp":1513807710949
}



Let's parse these JSON strings into JSON objects

In [13]:
try:
    high_rotaryAngle_JSON = json.loads(high_rotaryAngle_String)
    high_temperature_JSON = json.loads(high_temperature_String)
    low_temperature_JSON = json.loads(low_temperature_String)
except:
     print("Unexpected error:", sys.exc_info()[0])

In [14]:
high_rotaryAngle_Value = high_rotaryAngle_JSON['value']
print("The rotary angle value is %d." % high_rotaryAngle_Value)

high_temperature_Value = high_temperature_JSON['value']
print("The high temperature value is %d." % high_temperature_Value)

low_temperature_Value = low_temperature_JSON['value']
print("The lower temperature value is %d." % low_temperature_Value)

The rotary angle value is 600.
The high temperature value is 40.
The lower temperature value is 10.


## Use Helper functions to Execute a Rule's Action

This is the procedure
1. Filter the rules by sensorID
2. Check the predicate for each filtered rule
3. Execute the *action* function for each True predicate

#### For the **rotaryAngle** sensor data:

In [15]:
# Filter by sensor
filteredRules = filterBySensorId("rotaryAngle", rules)
print filteredRules

[<__main__.RotaryAngleOver500 instance at 0x104eb9170>]


In [16]:
# Execute the action function for each predicate that's True
[r.action(high_rotaryAngle_Value) 
  for r in filteredRules
    if r.predicate(high_rotaryAngle_Value) == True]

Rotary Angle Sensor over 500. Rule activated on rotaryAngle with senor value 600


[None]

#### For the High Temperature Sensor

In [17]:
# Filter by sensor
filteredRules = filterBySensorId("temperature", rules)
print filteredRules

[<__main__.TemperatureOver25 instance at 0x104eae8c0>, <__main__.TemperatureUnder20 instance at 0x104eb9368>]


In [18]:
# Execute the action function for each predicate that's True
[r.action(high_temperature_Value) 
  for r in filteredRules
    if r.predicate(high_temperature_Value) == True]

Temperature Over 25 Rule activated on temperature with senor value 40


[None]

#### For the Low Temperature Sensor

In [19]:
# Filter by sensor
filteredRules = filterBySensorId("temperature", rules)
print filteredRules

[<__main__.TemperatureOver25 instance at 0x104eae8c0>, <__main__.TemperatureUnder20 instance at 0x104eb9368>]


In [20]:
# Execute the action function for each predicate that's True
[r.action(low_temperature_Value) 
  for r in filteredRules
    if r.predicate(low_temperature_Value) == True]

Temperature Under 20 Rule activated on temperature with senor value 10


[None]

## Setup the MQTT subscription of the Automation Service

In [21]:
import paho.mqtt.client as mqtt

In [22]:
def on_connect(mqttc, obj, flags, rc):
    print("rc: " + str(rc))

In [23]:
def on_message(mqttc, obj, msg):
    print(msg.topic + " " + str(msg.qos) + " " + str(msg.payload))
    try:
        parsed_json = json.loads(msg.payload)
    except:
        print("Unexpected error:", sys.exc_info()[0])
    
    sensorId = parsed_json['sensor_id']
    sensorValue = parsed_json['value']
    
    filteredRules = filterBySensorId(sensorId, rules)
    
    [r.action(sensorValue) 
        for r in filteredRules
            if r.predicate(sensorValue) == True]


In [24]:
def on_subscribe(mqttc, obj, mid, granted_qos):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))

In [25]:
def on_log(mqttc, obj, level, string):
    print(string)

In [26]:
mqttc = mqtt.Client()
mqttc.on_message = on_message
mqttc.on_connect = on_connect
mqttc.on_subscribe = on_subscribe

# Uncomment to enable debug messages
# mqttc.on_log = on_log

In [27]:
mqttc.connect("localhost", 1883, 60)

0

In [28]:
mqttc.subscribe("sensors/+/data")

(0, 1)

In [ ]:
mqttc.loop_forever()

rc: 0
Subscribed: 1 (0,)
sensors/temperature/data 0 {"sensor_id":"temperature","value":24,"timestamp":1515801946988}
sensors/temperature/data 0 {"sensor_id":"temperature","value":21,"timestamp":1515801947991}
sensors/temperature/data 0 {"sensor_id":"temperature","value":20,"timestamp":1515801948994}
sensors/temperature/data 0 {"sensor_id":"temperature","value":27,"timestamp":1515801949997}
Temperature Over 25 Rule activated on temperature with senor value 27
sensors/temperature/data 0 {"sensor_id":"temperature","value":29,"timestamp":1515801951002}
Temperature Over 25 Rule activated on temperature with senor value 29
sensors/temperature/data 0 {"sensor_id":"temperature","value":19,"timestamp":1515801952008}
Temperature Under 20 Rule activated on temperature with senor value 19
sensors/temperature/data 0 {"sensor_id":"temperature","value":29,"timestamp":1515801953007}
Temperature Over 25 Rule activated on temperature with senor value 29
sensors/temperature/data 0 {"sensor_id":"temperatu